In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Importing required libraries
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf
from tensorflow.keras import layers
import keras_tuner as kt
from tensorflow import keras

# Importing specific modules from libraries
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Setting numpy print options to display floating point numbers up to 3 decimal points
np.set_printoptions(precision=3, suppress=True)


<IPython.core.display.Javascript object>

### 1. Load the datasets

##### leakage dataset train [m].csv
##### leakage dataset validation 1000.csv

and store the data as NumPy-Arrays
X train, Y train
X validation, Y validation.

In [3]:
# Loading the data from CSV files

# Reading the first dataset from a CSV file named "leakage_dataset_train_100.csv" and storing it in a pandas dataframe named df_100
df_100 = pd.read_csv("Data/leakage_dataset_train_100.csv", header=0)

# Reading the second dataset from a CSV file named "leakage_dataset_train_1000.csv" and storing it in a pandas dataframe named df_1000
df_1000 = pd.read_csv("Data/leakage_dataset_train_1000.csv", header=0)

# Reading the validation dataset from a CSV file named "leakage_dataset_validation_1000.csv" and storing it in a pandas dataframe named df_val
df_val = pd.read_csv("Data/leakage_dataset_validation_1000.csv", header=0)


<IPython.core.display.Javascript object>

In [4]:
# Load the features and label from dataframe and convert it to numpy array

# List the columns for the features and label for each dataframe
feature_cols = ['mfc1', 'mfc2', 'mfc3', 'mfc4']
label_cols = ['y1', 'y2']

# Load the 100-row training set and convert the features and label to numpy arrays
train_100 = df_100[feature_cols + label_cols].to_numpy()
X_train_100, y_train_100 = train_100[:, :4], train_100[:, 4:]

# Load the 1000-row training set and convert the features and label to numpy arrays
train_1000 = df_1000[feature_cols + label_cols].to_numpy()
X_train_1000, y_train_1000 = train_1000[:, :4], train_1000[:, 4:]

# Load the validation set and convert the features and label to numpy arrays
val = df_val[feature_cols + label_cols].to_numpy()
X_val, y_val = val[:, :4], val[:, 4:]


<IPython.core.display.Javascript object>

In [5]:
# # Calculate the mean and standard deviation of x1, x2, and x3 in the training data
# x1_mean = np.mean(X_train_1000[:, 0])
# x2_mean = np.mean(X_train_1000[:, 1])
# x3_mean = np.mean(X_train_1000[:, 2])
# x4_mean = np.mean(X_train_1000[:, 3])

# x1_std = np.std(X_train_1000[:, 0])
# x2_std = np.std(X_train_1000[:, 1])
# x3_std = np.std(X_train_1000[:, 2])
# x4_std = np.std(X_train_1000[:, 3])

<IPython.core.display.Javascript object>

In [6]:
# # Subtract the mean from x1, x2, and x3 in both the training and validation/test data
# X_train_1000[:, 0] = (X_train_1000[:, 0] - x1_mean) / x1_std
# X_train_1000[:, 1] = (X_train_1000[:, 1] - x2_mean) / x2_std
# X_train_1000[:, 2] = (X_train_1000[:, 2] - x3_mean) / x3_std
# X_train_1000[:, 3] = (X_train_1000[:, 3] - x4_mean) / x4_std

# X_train_100[:, 0] = (X_train_100[:, 0] - x1_mean) / x1_std
# X_train_100[:, 1] = (X_train_100[:, 1] - x2_mean) / x2_std
# X_train_100[:, 2] = (X_train_100[:, 2] - x3_mean) / x3_std
# X_train_100[:, 3] = (X_train_100[:, 3] - x4_mean) / x4_std

# X_val[:, 0] = (X_val[:, 0] - x1_mean) / x1_std
# X_val[:, 1] = (X_val[:, 1] - x2_mean) / x2_std
# X_val[:, 2] = (X_val[:, 2] - x3_mean) / x3_std
# X_val[:, 3] = (X_val[:, 3] - x4_mean) / x4_std

<IPython.core.display.Javascript object>

In [7]:
# # Calculate the mean and standard deviation of x1, x2, and x3 in the training data
# x_mean = np.mean(X_train_1000, axis=0)
# x_std = np.std(X_train_1000, axis=0)

# # Standardize the data
# X_train_1000 = (X_train_1000 - x_mean) / x_std
# X_train_100 = (X_train_100 - x_mean) / x_std
# X_val = (X_val - x_mean) / x_std

# # Normalize the data
# X_train_1000 = X_train_1000 / np.sum(X_train_1000, axis=1)[:, np.newaxis]
# X_train_100 = X_train_100 / np.sum(X_train_100, axis=1)[:, np.newaxis]
# X_val = X_val / np.sum(X_val, axis=1)[:, np.newaxis]

<IPython.core.display.Javascript object>

For that purpose, you have training data (xi, yi)
mi=1 available. Unfortunately,
the fourth sensor MFC4 had a slight malfunction at the time when the training data were produced. As a consequence, the values x4 are on average
higher than their counterparts x1, x2 and x3. Be aware that the same effect
is not present in the validation and test data.

In [8]:
# Preprocess the training data to account for the issue with the fourth sensor MFC4
X_train_100[:, 3] = X_train_100[:, 3] * (
    np.mean(X_train_100[:, :3], axis=1) / np.mean(X_train_100[:, 3])
)
X_train_1000[:, 3] = X_train_1000[:, 3] * (
    np.mean(X_train_1000[:, :3], axis=1) / np.mean(X_train_1000[:, 3])
)

<IPython.core.display.Javascript object>

### 2. Train a standard fully connected neural network. Hyperparameters

like number of epochs, batch size, optimizer, learning rate, depth and
width of the network, activation and loss functions – all up to you. You
can use a framework like KerasTuner to optimize hyperparameters, or
simple for-loops if you prefer that. Anyway, you should use the validation data in order to evaluate and compare candidate hyperparameter
configurations. Save the model as model standard [m].h5.

In [9]:
def standard_model(hp):
    model = Sequential()
    model.add(
        Dense(
            units=hp.Int("units_1", min_value=32, max_value=512, step=32),
            input_shape=(4,),
            activation=hp.Choice("activation_1", values=["relu", "tanh", "sigmoid"]),
        )
    )

    model.add(
        Dense(
            units=hp.Int(f"units_2", min_value=16, max_value=128, step=16),
            activation=hp.Choice(
                f"activation_2",
                values=["relu", "tanh", "sigmoid"],
            ),
        )
    )

    model.add(Dense(2, activation="softmax"))

    # Set optimizer and learning rate
    optimizer = hp.Choice("optimizer", values=["adam", "adagrad", "nadam", "rmsprop"])
    learning_rate = hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3, 1e-4])

    if optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "adagrad":
        optimizer = keras.optimizers.Adagrad(learning_rate=learning_rate)
    elif optimizer == "rmsprop":
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.Nadam(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )

    return model

<IPython.core.display.Javascript object>

In [10]:
# def with_all_standard_model(hp):
#     model = Sequential()
#     model.add(
#         Dense(
#             units=hp.Int("units_1", min_value=32, max_value=512, step=32),
#             input_shape=(4,),
#             activation=hp.Choice("activation_1", values=["relu", "tanh", "sigmoid"]),
#         )
#     )

#     model.add(Dropout(hp.Float("dropout_1", min_value=0.2, max_value=0.5, step=0.1)))
#     model.add(BatchNormalization())

#     model.add(
#         Dense(
#             units=hp.Int(f"units_2", min_value=16, max_value=128, step=16),
#             activation=hp.Choice(
#                 f"activation_2",
#                 values=["relu", "tanh", "sigmoid"],
#             ),
#         )
#     )

#     model.add(Dropout(hp.Float(f"dropout_2", min_value=0.2, max_value=0.5, step=0.1)))
#     model.add(BatchNormalization())

#     model.add(Dense(2, activation="softmax"))

#     # Set optimizer and learning rate
#     optimizer = hp.Choice("optimizer", values=["adam", "adagrad", "nadam", "rmsprop"])
#     learning_rate = hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3, 1e-4])

#     if optimizer == "adam":
#         optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
#     elif optimizer == "adagrad":
#         optimizer = keras.optimizers.Adagrad(learning_rate=learning_rate)
#     elif optimizer == "rmsprop":
#         optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
#     else:
#         optimizer = keras.optimizers.Nadam(learning_rate=learning_rate)

#     model.compile(
#         optimizer=optimizer,
#         loss="mean_squared_error",
#         metrics=["mean_absolute_error"],
#     )

#     # Add regularization
#     if hp.Boolean("use_regularization"):
#         l1_reg = hp.Float("l1_reg", min_value=1e-6, max_value=1e-2, sampling="log")
#         l2_reg = hp.Float("l2_reg", min_value=1e-6, max_value=1e-2, sampling="log")
#         model.add(keras.layers.ActivityRegularization(l1=l1_reg, l2=l2_reg))

#     return model

<IPython.core.display.Javascript object>

In [ ]:
tuner_100 = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=100,
    factor=3,
    overwrite=True,
    directory="my_dir",
    project_name="standard_model_100",
)
# Search for the best hyperparameters
early_stopping = EarlyStopping(monitor="val_loss", patience=10)
tuner_100.search(
    X_train_100,
    y_train_100,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
#     verbose=1,
)

Trial 217 Complete [00h 00m 05s]
val_loss: 0.5614820718765259

Best val_loss So Far: 0.42471805214881897
Total elapsed time: 00h 10m 54s

Search: Running Trial #218

Value             |Best Value So Far |Hyperparameter
224               |32                |units_1
tanh              |tanh              |activation_1
48                |32                |units_2
tanh              |tanh              |activation_2
rmsprop           |nadam             |optimizer
0.0001            |0.01              |learning_rate
12                |34                |tuner/epochs
0                 |12                |tuner/initial_epoch
2                 |3                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/12
1/4 [======>.......................] - ETA: 1s - loss: 0.6711 - mean_absolute_error: 0.6743

In [ ]:
best_hps_100 = tuner_100.get_best_hyperparameters(num_trials=1)[0]
model_100 = tuner_100.hypermodel.build(best_hps_100)

In [ ]:
# Train the model on the training set
history_100 = model_100.fit(
    X_train_100,
    y_train_100,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
)

In [ ]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model_100.evaluate(X_val, y_val)
print("Validation loss: {:.3f}".format(val_loss))
print("Validation accuracy: {:.3f}".format(val_acc))


# Save the model
model_100.save("models/model_standard_100.h5")

In [ ]:
tuner_1000 = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=100,
    factor=3,
    overwrite=True,
    directory="my_dir",
    project_name="standard_model_1000",
)
# Search for the best hyperparameters
early_stopping = EarlyStopping(monitor="val_loss", patience=10)
tuner_1000.search(
    X_train_1000,
    y_train_1000,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
#     verbose=1,
)


In [ ]:
best_hps_1000 = tuner_1000.get_best_hyperparameters(num_trials=1)[0]
model_1000 = tuner_1000.hypermodel.build(best_hps_1000)

In [ ]:
# Train the model on the training set
history_1000 = model_1000.fit(
    X_train_1000,
    y_train_1000,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
)

In [ ]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model_1000.evaluate(X_val, y_val)
print("Validation loss: {:.3f}".format(val_loss))
print("Validation accuracy: {:.3f}".format(val_acc))


# Save the model
model_1000.save("models/model_standard_1000.h5")

### 3. Augment the dataset. 

Each training example (x, y) can be complemented by seven additional virtual examples as illustrated in Figure 1.
Clockwise rotations and flips on input data x are obtained as described
above. A clockwise rotation on output data y is obtained through
y90 = (y2, −y1) and a flip along the vertical axis on the same data is
represented by yf lipped = (−y1, y2). All other operations (rotation angles and associated flips) can be computed by subsequent application
of these two operations.

In [ ]:
def augment_data(x_train, y_train):
    # initialize empty lists for augmented data and labels
    x_train_aug = []
    y_train_aug = []

    for i in range(len(x_train)):
        x_train_aug.append(x_train[i])
        y_train_aug.append(y_train[i])

        # 90 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array([-x_train[i][1], x_train[i][0], -x_train[i][3], x_train[i][2]])
            )
        )
        y_train_aug.append(np.flip(np.array([y_train[i][1], -y_train[i][0]])))

        # 180 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array([-x_train[i][2], -x_train[i][3], x_train[i][0], x_train[i][1]])
            )
        )
        y_train_aug.append(np.flip(np.array([-y_train[i][0], -y_train[i][1]])))

        # 270 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array([x_train[i][3], -x_train[i][2], x_train[i][1], -x_train[i][0]])
            )
        )
        y_train_aug.append(np.flip(np.array([-y_train[i][1], y_train[i][0]])))

        # vertical flip
        x_train_aug.append(
            np.flip(
                np.array([x_train[i][2], x_train[i][3], x_train[i][0], x_train[i][1]])
            )
        )
        y_train_aug.append(np.flip(np.array([-y_train[i][0], y_train[i][1]])))

        # horizontal flip + 90 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array(
                    [-x_train[i][1], -x_train[i][0], -x_train[i][3], -x_train[i][2]]
                )
            )
        )
        y_train_aug.append(np.flip(np.array([-y_train[i][1], y_train[i][0]])))

        # horizontal flip + 180 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array([-x_train[i][2], -x_train[i][1], x_train[i][0], x_train[i][3]])
            )
        )
        y_train_aug.append(np.flip(np.array([y_train[i][0], y_train[i][1]])))

        # horizontal flip + 270 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array([x_train[i][3], x_train[i][2], -x_train[i][1], x_train[i][0]])
            )
        )
        y_train_aug.append(np.flip(np.array([y_train[i][1], -y_train[i][0]])))
    return x_train_aug, y_train_aug

In [ ]:
# Augment the training data with the augment_data function for the X_train_100 and y_train_100 data
x_train_augmented, y_train_augmented = augment_data(X_train_100, y_train_100)
x_train_augmented_1000, y_train_augmented_1000 = augment_data(
    X_train_1000, y_train_1000
)

# Convert the augmented data and labels to numpy arrays
X_train_augmented = np.array(x_train_augmented)
y_train_augmented = np.array(y_train_augmented)
X_train_augmented_1000 = np.array(x_train_augmented_1000)
y_train_augmented_1000 = np.array(y_train_augmented_1000)

### 4. Repeat 2. 
on the augmented dataset. Save this model as model standard [m] augmented.h5.

In [ ]:
tuner_100_aug = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=100,
    factor=3,
    overwrite=True,
    directory="my_dir",
    project_name="standard_model_aug_100",
)
# Search for the best hyperparameters
early_stopping = EarlyStopping(monitor="val_loss", patience=10)
tuner_100_aug.search(
    X_train_augmented,
    y_train_augmented,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
#     verbose=1,
)

In [ ]:
best_hps_aug_100 = tuner_100_aug.get_best_hyperparameters(num_trials=1)[0]
model_aug_100 = tuner_100_aug.hypermodel.build(best_hps_aug_100)

In [ ]:
# Train the model on the training set
history_aug_100 = model_aug_100.fit(
    X_train_augmented,
    y_train_augmented,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
)

In [ ]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model_aug_100.evaluate(X_val, y_val)
print("Validation loss: {:.3f}".format(val_loss))
print("Validation accuracy: {:.3f}".format(val_acc))


# Save the model
model_aug_100.save("models/model_standard_100_augmented.h5")

In [ ]:
tuner_1000_aug = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=100,
    factor=3,
    overwrite=True,
    directory="my_dir",
    project_name="standard_model_aug_1000",
)
# Search for the best hyperparameters
early_stopping = EarlyStopping(monitor="val_loss", patience=10)
tuner_1000_aug.search(
    X_train_augmented_1000,
    y_train_augmented_1000,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
#     verbose=1,
)

In [ ]:
best_hps_aug_1000 = tuner_1000_aug.get_best_hyperparameters(num_trials=1)[0]
model_aug_1000 = tuner_1000_aug.hypermodel.build(best_hps_aug_1000)

In [ ]:
# Train the model on the training set
history_aug_1000 = model_aug_1000.fit(
    X_train_augmented_1000,
    y_train_augmented_1000,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
)

In [ ]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model_aug_1000.evaluate(X_val, y_val)
print("Validation loss: {:.3f}".format(val_loss))
print("Validation accuracy: {:.3f}".format(val_acc))


# Save the model
model_aug_100.save("models/model_standard_1000_augmented.h5")